In [1]:
! pip install -U tqdm

import copy
import time
import torch
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

print("PyTorch Version: ", torch.__version__)


Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.43.0)
PyTorch Version:  1.4.0


In [15]:
! cp -r '/content/drive/My Drive/TEMP/gala_dataset.zip' /content/
! unzip /content/gala_dataset.zip  -d /content/

Archive:  /content/gala_dataset.zip
replace /content/dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/dataset/Test Images/image9683.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace /content/dataset/Test Images/image9683.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
'''
set some configuration variables
'''

valid_model_names = ['resnet', 'alexnet']

train_csv_path = '/content/dataset/train.csv'
test_csv_path = '/content/dataset/test.csv'
train_images_dir = '/content/dataset/Train Images/'
test_images_dir = '/content/dataset/Test Images/'


model_name = 'resnet'

num_classes = 4

batch_size = 64

num_epochs = 3

# set val_split = 0 for no validation
val_split = 0.0
# if dev_mode = True loads only a few samples
dev_mode = False

# feature_extract = False   ==> fine-tune the whole model 
# feature_extract = True    ==> only update the reshaped layer parameters
feature_extract = False


In [0]:
'''
Define Transforms
Define Dataset Class
'''

class galaDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        self.df = df
        self.image_dir = image_dir
        self.transforms = transforms
        self.label_map = {'Decorationandsignage':0,
                          'Food':1,
                          'misc':2,
                          'Attire':3}
        try:
            self.classes = df.Class.unique()
            self.train = True
        except:
            self.train = False

    def __len__(self):
        return self.df.__len__()
    
    def __getitem__(self, idx):
        image_path = self.image_dir + self.df.loc[idx, 'Image']
        image = Image.open(image_path).convert('RGB')
        
        if self.transforms:
            image = self.transforms(image)

        if self.train:
            # if its a train/val dataset
            label = self.label_map[self.df.loc[idx, 'Class']]
            return image, label
        else:
            # if its a test dataset
            return image


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 244)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAffine(degrees=[0,10]),
        # transforms.RandomPerspective(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    'val': transforms.Compose([
        transforms.Resize((224, 244)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
    'test': transforms.Compose([
        transforms.Resize((224, 244)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
}


In [0]:
'''
Prepare Dataset
'''

if dev_mode:
    tr_df = pd.read_csv(train_csv_path).sample(100)
else:
    tr_df = pd.read_csv(train_csv_path)


if val_split > 0:
    tr_df, val_df = train_test_split(tr_df, test_size = val_split)
    val_df = val_df.reset_index(drop=True)
    tr_df = tr_df.reset_index(drop=True)
    val_dataset = galaDataset(val_df, train_images_dir, data_transforms['val'])


tr_dataset = galaDataset(tr_df, train_images_dir, data_transforms['train'])

te_dataset = galaDataset(pd.read_csv(test_csv_path), test_images_dir, data_transforms['test'])


if val_split > 0:
    datasets_dict = {'train' : tr_dataset, 'test' : te_dataset, 'val' : val_dataset }
else:
    datasets_dict = {'train' : tr_dataset, 'test' : te_dataset,}



In [0]:
'''
train_model parameters:
    pytorch model
    dictionary of dataset objects
      ==> if validation dataset is passed with key 'val', validation is performer, else skipped
    criterion : loss function
    optimizer
    number of epochs to train and validate for
'''

def train_model(model, datasets_dict, criterion, optimizer, batch_size, num_epochs = 25):
    
    since = time.time()

    do_validation = (datasets_dict.get('val') != None)
    
    train_acc_hist = []
    train_loss_hist = []

    if do_validation:
        val_acc_history = []
        val_f1_history = []
        val_dataloader = DataLoader(datasets_dict['val'], batch_size=batch_size, shuffle=True)
        print('Validating on {} samples.'.format(datasets_dict['val'].__len__()))
        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0


    for epoch in range(num_epochs):

        tr_dataloader = DataLoader(datasets_dict['train'], batch_size=batch_size, shuffle=True, num_workers=4)

        if do_validation:
            val_preds = []
            val_labels = []
            phases = ['train', 'val']
            dataloaders = {'train' : tr_dataloader, 'val' : val_dataloader}
        else:
            phases = ['train']
            dataloaders = {'train' : tr_dataloader}


        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-'*10)

        for phase in phases:
            if phase == 'train':    model.train()
            elif phase == 'val':    model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                model.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if do_validation:
                    val_preds.append(preds.cpu())
                    val_labels.append(labels.cpu())

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if phase == 'train':
                train_acc_hist.append(epoch_acc)
                train_loss_hist.append(epoch_loss)

        if do_validation:
            val_f1 = get_f1(val_preds, val_labels)
            print("val F1 : ", val_f1)
            val_f1_history.append(val_f1)

        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    if do_validation:
        print('Best val Acc: {:4f}'.format(best_acc))
        model.load_state_dict(best_model_wts)
        metrics = {'val_acc_hist':val_acc_history, 'val_f1_hist':val_f1_history, 'train_acc_hist':train_acc_hist, 'train_loss_hist':train_loss_hist}
    else:
        metrics = {'train_acc_hist':train_acc_hist, 'train_loss_hist':train_loss_hist}
    
    return model, metrics


def set_parameters_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, features_extract, use_pretrained=True):
    model_ft = None
    input_size = 0
    
    _ = ['resnet', 'alexnet']
    
    if model_name == 'resnet':
        model_ft = models.resnet152(pretrained=use_pretrained)
        set_parameters_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameters_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    else:
        print('Invalid model name, exiting. . .') 
        exit()
    
    return model_ft, input_size


def get_f1(val_preds, val_labels):
    VP = []
    VL = []
    for l in [list(x.numpy()) for x in val_preds]: VP.extend(l)
    for l in [list(x.numpy()) for x in val_labels]: VL.extend(l)
    return f1_score(VL, VP, average='weighted')


In [0]:
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    model_ft.cuda()
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model_ft = model_ft.to(device)


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [17]:

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name, end='')
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name, end='')


print()


optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)

criterion = nn.CrossEntropyLoss()


Params to learn:
	 conv1.weight	 bn1.weight	 bn1.bias	 layer1.0.conv1.weight	 layer1.0.bn1.weight	 layer1.0.bn1.bias	 layer1.0.conv2.weight	 layer1.0.bn2.weight	 layer1.0.bn2.bias	 layer1.0.conv3.weight	 layer1.0.bn3.weight	 layer1.0.bn3.bias	 layer1.0.downsample.0.weight	 layer1.0.downsample.1.weight	 layer1.0.downsample.1.bias	 layer1.1.conv1.weight	 layer1.1.bn1.weight	 layer1.1.bn1.bias	 layer1.1.conv2.weight	 layer1.1.bn2.weight	 layer1.1.bn2.bias	 layer1.1.conv3.weight	 layer1.1.bn3.weight	 layer1.1.bn3.bias	 layer1.2.conv1.weight	 layer1.2.bn1.weight	 layer1.2.bn1.bias	 layer1.2.conv2.weight	 layer1.2.bn2.weight	 layer1.2.bn2.bias	 layer1.2.conv3.weight	 layer1.2.bn3.weight	 layer1.2.bn3.bias	 layer2.0.conv1.weight	 layer2.0.bn1.weight	 layer2.0.bn1.bias	 layer2.0.conv2.weight	 layer2.0.bn2.weight	 layer2.0.bn2.bias	 layer2.0.conv3.weight	 layer2.0.bn3.weight	 layer2.0.bn3.bias	 layer2.0.downsample.0.weight	 layer2.0.downsample.1.weight	 layer2.0.downsample.1.bias	 layer2.1.conv

In [24]:
# run the training loop
model_ft, hist = train_model(model_ft, datasets_dict, criterion, optimizer_ft, batch_size, num_epochs=1)

Epoch 1/1
----------



train Loss: 0.1439 Acc: 0.9544

Training complete in 1m 19s


In [24]:
'''
Pickle model and training metrics on drive
'''
# import pickle
# with open('/content/drive/My Drive/TEMP/gala_image_tagging_hist.pickle', 'wb') as f:
#     pickle.dump(hist, f)
# with open('/content/drive/My Drive/TEMP/gala_image_tagging_model.pickle', 'wb') as f:
#     pickle.dump(model_ft, f)


'\nPickle model and training metrics on drive\n'

In [0]:
'''
Un-Pickle saved entities
'''
# import pickle
# with open('/content/drive/My Drive/TEMP/gala_image_tagging_val_accu_hist.pickle', 'rb') as f:
#   hist = pickle.load(f)
# with open('/content/drive/My Drive/TEMP/gala_image_tagging_model.pickle', 'rb') as f:
#   model_ft = pickle.load(f)


# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())
#     print('We will use the GPU:', torch.cuda.get_device_name(0))
#     model_ft.cuda()
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")

# model_ft = model_ft.to(device)


#**GET TEST PREDICTIONS** 

In [25]:
print('Generating predictions for {} samples'.format(te_dataset.__len__()))

te_dataloader = DataLoader(te_dataset, batch_size=batch_size, shuffle=False)

model_ft.eval()
test_preds = None

for inputs in tqdm(te_dataloader):

    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model_ft(inputs)

        if test_preds is None:
            test_preds = outputs.data.cpu()
        else:
            test_preds = torch.cat((test_preds, outputs.data.cpu()), dim=0)
    
res = test_preds.argmax(dim=1)
te_df = pd.read_csv(test_csv_path)
te_df['Class'] = res

label_map = {
    0:'Decorationandsignage',
    1:'Food',
    2:'misc',
    3:'Attire'}
te_df['Class'] = te_df.Class.map(label_map)


te_df.to_csv('/content/gala_submission_resnet152_6-epoch_64-batch_lesser-aug.csv', index=False)
hist

Generating predictions for 3219 samples


{'train_acc_hist': [tensor(0.9544, device='cuda:0', dtype=torch.float64)],
 'train_loss_hist': [0.14388068470392223]}

cp: cannot stat '/content/gala_submission_resnet152_3-epoch_64-batch_lesser-aug.csv': No such file or directory
